# Upload Structure

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
from aiida.orm import load_node


from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.cif import CifData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.calculation import Calculation
from aiida.common.links import LinkType

from aiida.work import workfunction as wf


import re
import csv
import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

## Step 1: Upload file

In [ ]:
folder_db = '/project/materials-cloud'

In [ ]:
start_from = 1
end_at = None

In [ ]:
prms_dict = {}

In [ ]:
@wf
def link_parameters(cif):
    return ParameterData(dict=prms_dict)

In [ ]:
def upload_all_structures():
    with open(folder_db+'/output.allStructures', 'r') as csvfile:
        global prms_dict
        number_of_structures = 1
        prog = re.compile('\[(.*?)\]')
        reader = csv.reader(csvfile)

        fields = next(reader, None)
        for i,unit in enumerate(fields): 
            fields[i]= unit.strip()

        name_fld = [i for i,x in enumerate(fields) if x == 'name'][0]

        for row in reader:
            if number_of_structures < start_from and start_from is not None:
                number_of_structures += 1
                continue
            if number_of_structures > end_at and end_at is not None:
                break

            prms_dict = {}
            for i,unit in enumerate(row): 
                row[i]= unit.strip()
            cif = CifData(file =folder_db+'/structures/'+row[name_fld]+'.cif', scan_type='flex', parse_policy='lazy')

            cif.description = row[name_fld]

            for i,fld in enumerate(fields):
                m = re.search(prog, fld)
                if m:
                    units = m.group(1).strip()
                    fld = re.sub(prog,'',fld).strip().replace(' ', '_')
                    prms_dict[fld] = row[i]
                    prms_dict[fld+'_units'] = units
                else:
                    fld = fld.replace(' ', '_')
                    prms_dict[fld] = row[i]

            params = link_parameters(cif=cif)
            number_of_structures += 1
        print (number_of_structures-1)


In [ ]:
#upload_all_structures()